In [ ]:
import torch
from torchmetrics.multimodal import CLIPImageQualityAssessment
from torchvision import transforms
from PIL import Image
from pathlib import Path
import os
import numpy as np

# 1. CLIP 평가 모델 로드
device = "cuda" if torch.cuda.is_available() else "cpu"
gpu_number = 7
if torch.cuda.is_available():
    torch.cuda.set_device(gpu_number)
print(f"Using {device} device")
clip_metric = CLIPImageQualityAssessment(model_name_or_path="openai/clip-vit-base-patch16", prompts=("quality", "brightness")).to(device)

# clip_vision_encoder = clip_metric.model.vision_model

# clip_metric


Using cuda device


CLIPImageQualityAssessment(
  (model): CLIPModel(
    (text_model): CLIPTextTransformer(
      (embeddings): CLIPTextEmbeddings(
        (token_embedding): Embedding(49408, 512)
        (position_embedding): Embedding(77, 512)
      )
      (encoder): CLIPEncoder(
        (layers): ModuleList(
          (0-11): 12 x CLIPEncoderLayer(
            (self_attn): CLIPSdpaAttention(
              (k_proj): Linear(in_features=512, out_features=512, bias=True)
              (v_proj): Linear(in_features=512, out_features=512, bias=True)
              (q_proj): Linear(in_features=512, out_features=512, bias=True)
              (out_proj): Linear(in_features=512, out_features=512, bias=True)
            )
            (layer_norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
            (mlp): CLIPMLP(
              (activation_fn): QuickGELUActivation()
              (fc1): Linear(in_features=512, out_features=2048, bias=True)
              (fc2): Linear(in_features=2048, out_features=

In [ ]:


def image_tensor(image_path, size=None):
    img = Image.open(image_path)
    
    if size != None:
        img = img.resize(size)

    img = (np.asarray(img)/255.0)
    img = torch.from_numpy(img).float()
    img = img.permute(2,0,1).unsqueeze(0)
    return img

# 3. 이미지 로드 및 전처리
file_path = Path('./data/Huawei')
img_labels = sorted(os.listdir(file_path / 'low'))
lq_img = image_tensor(file_path / 'low' / img_labels[0], (224, 224)).to(device)
# image = transform(lq_img).unsqueeze(0).to(device)  # 배치 차원 추가

# 4. CLIP 특징 벡터 추출
with torch.no_grad():
    image_features = clip_vision_encoder(lq_img)

# 5. 출력 결과
print("Base Model Output With Pooling:", image_features[0].shape)  # 출력 벡터 차원 확인
print("Pooler Output:", image_features[1].shape)  # 출력 벡터 확인



Base Model Output With Pooling: torch.Size([1, 197, 768])
Pooler Output: torch.Size([1, 768])
